In [1]:
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
# polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:4326")

In [2]:
# gg= he.convert_list_attr_to_str(g.graph)
# for node,e,data in gg.edges(data=True):
#     if 'geometry' in data:
#         data['geometry']=str(data['geometry'])
# nx.write_graphml(gg, f"tula.graphml")

In [1]:
import sys
sys.path.append('/Users/polina/Desktop/github/transport_frames')
from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he
from transport_frames.indicators.indicator_terr import indicator_territory
from transport_frames.indicators.indicator_area import indicator_area
from transport_frames.indicators.utils import assign_services_names_to_nodes, availability_matrix
import osmnx as ox
import geonetworkx as gnx
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import momepy
import shapely
from dongraphio import GraphType


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## create graph and frame

In [2]:
# polygon regions cities
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

polygon = ox.geocode_to_gdf('R81993',by_osmid=True)

cities = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
cities = cities[cities.is_admin_centre_district==1][['name','geometry']].reset_index()

In [3]:
local_crs = 32637

In [4]:
# g = Graph.from_polygon(polygon, crs=local_crs)

tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/transport_frames/tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

2024-09-06 17:03:07.293 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:135 - Preparing the graph...
2024-09-06 17:03:17.594 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:173 - Graph is ready!


In [5]:
f = Frame(g.graph, regions, polygon, cities)

In [ ]:
f.get_geopackage()

## grade terr (by frame)

In [8]:
poly = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')

In [9]:
graded_terr = f.grade_territory(poly, include_priority=True)

## grade terr (advanced)

In [10]:
points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')
inter = he.prepare_graph(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_inter.graphml'))
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')

In [11]:
adj_mx_drive = availability_matrix(g.graph, points)
adj_mx_inter = availability_matrix(inter, points,graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK])

2024-09-06 16:50:10.995 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-09-06 16:50:14.965 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...
2024-09-06 16:53:07.461 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!
2024-09-06 16:53:07.470 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:84 - Creating adjacency matrix based on provided graph...
2024-09-06 16:53:16.021 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances ...
2024-09-06 16:56:25.899 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!


In [12]:
grader = AdvancedGrader(local_crs)
cri = grader.get_criteria(graded_terr=graded_terr,points=points,polygons=settlements,r_stops=r_stops,b_stops=b_stops,aero=aero,adj_mx_drive=adj_mx_drive,adj_mx_inter=adj_mx_inter)


In [13]:
grader.interpret_gdf()

[('Манаенское',
  ['Территория расположена в непосредственной близости от одной из региональной трассы (в радиус 5 км от границ территории попадает хотя бы 1 узел региональной трассы, которая является приоритетной, а ближайший федеральный узел находится в не более чем 10км);',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на личном транспорте;',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на общественном транспорте;',
   'В радиусе 15 км отсутствуют ЖД станции, порты/причалы/переправы, аэродромы.']),
 ('Астаповское',
  ['Территория расположена в непосредственной близости от одной из региональной трассы (в радиус 5 км от границ территории попадает хотя бы 1 узел региональной трассы, которая является приоритетной, а ближайший федеральный узел находится в не более чем 10км);',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на личном транспорте;',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на общественном транспорт

## indicators terr

In [14]:
# inter = nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_inter.graphml')
# inter = he.prepare_graph(inter)
poly = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson').reset_index()
settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')

b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
regions_gdf = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/district.geojson')

In [15]:
custom_territory=poly.iloc[[5]]
custom_territory_center = custom_territory.geometry.representative_point()
custom_territory_center = gpd.GeoDataFrame([{'geometry': custom_territory_center.iloc[0]}], crs=custom_territory_center.crs).to_crs(32637)


In [16]:
PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # если нет данных о сервисе

railway_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
railway_roads = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway.geojson')
bus_stops = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
bus_routes = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_bus_routes.geojson')
fuel_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия fuel.geojson')
ferry_terminal = PLACEHOLDER
local_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
# international_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_international_aerodrome.geojson')
# либо:
international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# Природные объекты
water = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия water.geojson')
nature_reserve = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия nature_reserve.geojson')


In [17]:
services = {
    'railway_stops' : railway_stations,
    'fuel' : fuel_stations,
    'ports' : ferry_terminal,
    'local_aero' : local_aerodrome,
    'international_aero' : international_aerodrome,
    'oopt' : nature_reserve,
    'water_objects' : water,
    'train_paths' : railway_roads,
    'bus_stops' : bus_stops,
    'capital': ox.geocode_to_gdf('N34389350',by_osmid=True),
    'points': points,
    'bus_routes' : bus_routes,
    'region_centers': cities,
    'neud_center': custom_territory_center

}

In [18]:
G = assign_services_names_to_nodes(services,momepy.nx_to_gdf(g.graph)[0],g.graph,crs = 32637)

railway_stops
fuel
no ports (((((((
local_aero
no international_aero (((((((
no oopt (((((((
no water_objects (((((((
no train_paths (((((((
bus_stops
capital
points
bus_routes
region_centers
neud_center


In [19]:
t = indicator_territory(G,poly.iloc[[5]],3857,regions_gdf, points, cities, services, *momepy.nx_to_gdf(g.graph))
t

100%|██████████| 1/1 [00:00<00:00,  8.36it/s]


nearest_region_centers
nearest_settlement_centers


,name,to_region_admin_center,to_reg1,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,water_objects_accessibility,oopt_accessibility,...,number_of_international_aero,number_of_bus_stops,number_of_water_objects,number_of_oopt,density,train_path_length,number_of_bus_routes,to_nearest_settlement_center,to_nearest_mo_center,geometry
29800,None,94.138,2.132,0,0,100.492,None,None,0.002,2.716,...,0,12,11,0,4444.598,19.209,64,1757.859,1757.859,"POLYGON ((36.08847 53.81940, 36.09122 53.81675..."


In [20]:
inter = he.prepare_graph(inter)

In [21]:
b = indicator_area(G, inter, services, [settlements])

TypeError: indicator_area() missing 1 required positional argument: 'local_crs'

In [ ]:
b[0]